# MS Calculations

Notebook to crunch numbers for the MS.

by Cascade Tuholske 2020.02.23 

Updated 2020.08.27 - CPT  
Updated July/Aug 2021

In [ ]:
#### Depdencies 
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
import glob
import matplotlib.dates as mdates
import os

In [ ]:
#### Regressions, no intercept addition is needed because we're using SK LEARN HERE 

def lm_func(df, col):
    
    "simple linear model of a time series data, returns coef"
    
    # Get Data
    X_year = np.array(df.groupby('year')['ID_HDC_G0'].mean().index).reshape((-1, 1))
    Y_stats = np.array(df.groupby('year')[col].sum()).reshape((-1, 1))

    # Add Intercept
    X_year_2 = sm.add_constant(X_year)

    # Regress
    model = sm.OLS(Y_stats, X_year_2).fit() 
        
    coef = int(model.params[1])
    #coef = int(coef)
            
    # R2 and P
    r2 = model.rsquared_adj
    p = model.pvalues[0]
    
    return coef, round(r2, 2), round(p, 3)

In [ ]:
#### Load Data
DATA = 'wbgtmax30' # UPDATE 

# file paths
DATA_IN = ""  
FIG_OUT = ""
FN_IN = os.path.join(DATA_IN,DATA+'_EXP.json')
HI_STATS = pd.read_json(FN_IN, orient = 'split')

# Set scale
scale = 10**9


In [ ]:
# Drop cites where 1983 had 1 day and none elsewhere

print(len(HI_STATS))
only83 = HI_STATS.groupby('ID_HDC_G0')['tot_days'].sum() == 1 # sum up total days and find those with 1 day
only83 = list(only83[only83 == True].index) # make a list of IDs
sub = HI_STATS[HI_STATS['ID_HDC_G0'].isin(only83)] # subset those IDs
bad_ids = sub[(sub['year'] == 1983) & (sub['tot_days'] == 1)] # drop those from 1983 only
drop_list = list(bad_ids['ID_HDC_G0']) # make a list
HI_STATS= HI_STATS[~HI_STATS['ID_HDC_G0'].isin(drop_list)] # drop those from the list
print(len(HI_STATS))

In [ ]:
#### Add In Meta Data (e.g. geographic data)
meta_fn = os.path.join('','GHS-UCDB-IDS.csv')
meta_data = pd.read_csv(meta_fn)

#### Merge in meta
HI_STATS = HI_STATS.merge(meta_data, on = 'ID_HDC_G0', how = 'left')

In [ ]:
 HI_STATS

In [ ]:
#### Check to num days per year greatest > 32
check = HI_STATS.sort_values('tot_days', ascending = False)
check[['CTR_MN_NM', 'UC_NM_MN', 'year', 'tot_days']].head(5)

# Global Trends

In [ ]:
#### Total Change in people Days
data = HI_STATS.groupby('year')['people_days'].sum()
year = str(data.index[33])
value = str(data.values[33]/10**9)
print('person days in 2016 was '+value+' billion')

year = str(data.index[0])
value = str(data.values[0]/10**9)
print('person days in 1983 was '+value+' billion')

#### Pct Change in Poeple Days 1983 - 2016
pdays16 = data.iloc[len(data) -1]
pdays83 = data.iloc[0]
out = (data.iloc[len(data) -1] - data.iloc[0]) / data.iloc[0] * 100
print('pct increase in people days 83 - 16 is ', out)


In [ ]:
#### Rate of change
data = HI_STATS
coef, r2, p = lm_func(data, 'people_days')
print('annual increase in people days ', 'was', coef/10**9, ' p=', p)
coef1, r21, p1 = lm_func(data, 'people_days_heat')
print('annual increase in people days heat ', 'was', coef1/10**9, ' p=', p)
coef2, r22, p2 = lm_func(data, 'people_days_pop')
print('annual increase in people days pop ', 'was', coef2/10**9, ' p=', p)
print('attrib heat ', 'was', coef1 / coef *100, ' p=', p, '\n')

In [ ]:
#### Pct Pday Annual Increase from Heat
coef_pdays, r2_pdays, p_pdays = lm_func(HI_STATS, 'people_days') # regress pdays
coef_heat, r2_heat, p_heat = lm_func(HI_STATS, 'people_days_heat') # regreas heat

print('warming is what pct of total?', coef_heat/coef_pdays *100)

In [ ]:
#### Pct heat vs pop
coef_pop, r2_pdays, p_pdays = lm_func(HI_STATS, 'people_days_pop') # regress pdays
coef_heat, r2_heat, p_heat = lm_func(HI_STATS, 'people_days_heat') # regreas heat

print('pct of heat vs pop', coef_heat/coef_pop *100)

# City-level

#### Largest cities compared to global total

In [ ]:
#### Top cities
cities_fn = os.path.join('','wbgtmax30_EXP-TOP50.csv')
cities = pd.read_csv(cities_fn)

In [ ]:
top = cities.sort_values('coef_pdays', ascending = False).head(25) # get the top ten cities


In [ ]:
# What pct of the global annual increase comes from the top 25 cities?
ans = top['coef_pdays'].sum() / coef # coef comes from rate of change cell
print('Top 25 cities of total annual increase', ans * 100)

#### Pdays

In [ ]:
city_coefs_fn = os.path.join('','wbgtmax30_TREND_PDAYS05.json')
city_coefs = pd.read_json(city_coefs_fn, orient = 'split')
GHS = gpd.read_file('','GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_0.shp')

In [ ]:
len(city_coefs)

In [ ]:
#### Number of cities w/ sig increase in exposure?
print('The pct of cities w/ increases in exposure: ', len(city_coefs)/len(GHS)*100)


In [ ]:
len(city_coefs)

In [ ]:
ans = len(city_coefs[(city_coefs['GCPNT_LAT'] < 23.5) & (city_coefs['GCPNT_LAT'] > -23.5)]) / len(city_coefs)
print('what pct of pday cities are low lat?', ans*100)

In [ ]:
print('what pct of global pop are cities with sig pdays?')

In [ ]:
def country_search(country, data_set):
    "what pct of cities had a p-day increase?"
    print('Num of Cities in '+country+' ', len(data_set[data_set['CTR_MN_NM'] == country]) / len(GHS[GHS['CTR_MN_NM'] == country]) *100)

In [ ]:
data_set = city_coefs

In [ ]:
country_search('Senegal', data_set)

In [ ]:
country_search('Nigeria', data_set)

In [ ]:
country_search('India', data_set)

#### Pct of global population exposured

In [ ]:
city_coefs_fn = os.path.join('','wbgtmax30_TREND_PDAYS05.json')
city_coefs = pd.read_json(city_coefs_fn, orient = 'split')

In [ ]:
pop_fn = os.path.join('','GHS-UCDB-Interp.csv')
pop = pd.read_csv(pop_fn)

p16 = pop[['ID_HDC_G0', 'P2016']]

In [ ]:
len(p16)

In [ ]:
pdays_pop = pd.merge(city_coefs[['ID_HDC_G0']], p16, on = 'ID_HDC_G0', how = 'inner')

In [ ]:
ans = pdays_pop['P2016'].sum() / p16['P2016'].sum() * 100 
print('What is the global urban population in 2016', p16['P2016'].sum() / 10**9)
print('How many people live in cities with increasing exp in 2016', pdays_pop['P2016'].sum() / 10**9)
print('What pct of total urban pop has sig increase exp in 2015', ans)

In [ ]:
# From UN-DESA 2018 estimates for total global pop in 2015
ans =  pdays_pop['P2016'].sum() / 7383009000 * 100
print('What pct of total world pop has sig increase exp in 2016', ans)

In [ ]:
# UN-DESA Urban pop in 2015 was  3 981 498
p16['P2016'].sum()

#### Total Heat Days 

In [ ]:
city_totdays_fn = os.path.join('', 'wbgtmax30_TREND_HEATP05.json')
city_totdays = pd.read_json(city_totdays_fn, orient = 'split')

In [ ]:
print('What pct of all cities had sig increase in days/yr > WBGT 30 C ?')
print(len(city_totdays)/len(GHS))
print(len(city_totdays))

In [ ]:
print('What pct of all cities >1 day / yr in days/yr > WBGT 30 C ?')
print(len(city_totdays[city_totdays['coef_totDays'] >= 1])/len(GHS))
print(len(city_totdays[city_totdays['coef_totDays'] >= 1]))

In [ ]:
## How many cities day increase per year ... 1, 3
top = len(city_totdays)
bottom = len(city_totdays[city_totdays['coef_totDays'] >= 2])

In [ ]:
print(top)
print(bottom)

In [ ]:
## What are some big cities (>1m people)?
hot1m = city_totdays[(city_totdays['coef_totDays'] >= 1.5) & (city_totdays['P2016'] >= 10**6)][['coef_totDays', 'UC_NM_MN', 'P2016']].sort_values('coef_totDays')

In [ ]:
len(hot1m)

In [ ]:
hot1m

In [ ]:
2.162029e+07 / 10**6


#### Dehli & Kolkata 

In [ ]:
# Delhi 6955 & Kolkata 9691
K = city_coefs[city_coefs['ID_HDC_G0']== 9691]
D = city_coefs[city_coefs['ID_HDC_G0']== 6955]

In [ ]:
print('Share of heat Kolkata', K.coef_heat / K.coef_pdays * 100)

In [ ]:
print('Share of heat Delhi', D.coef_heat / D.coef_pdays * 100)

#### Populations of specific cities

In [ ]:
pop = pd.read_csv(DATA_IN+'interim/GHS-UCDB-Interp.csv')

In [ ]:
# 9691, Kolkata 1998
# 2046, Paris 2003
# 4417, Aleppo 2010

In [ ]:
ans = pop[pop['ID_HDC_G0'] == 9691]['P2015'] / 10**3
print('Pop of Kolkata in 1998', ans)

## WBGT 32 vs 28

In [ ]:
wbgt32 = pd.read_json('', '/wbgtmax32_TREND_PDAYS05.json', orient = 'split')
wbgt30 = pd.read_json('', '/wbgtmax30_TREND_PDAYS05.json', orient = 'split')
wbgt28 = pd.read_json('', '/wbgtmax28_TREND_PDAYS05.json', orient = 'split')

In [ ]:
print('how many cities wbgt 32?', len(wbgt32))
print('how many cities wbgt 32?', len(wbgt30))
print('how many cities wbgt 28?', len(wbgt28))
print('no trend', len(meta_data) - len(wbgt28))
print('dif is', 8510 - 6022)

# Regional Trends

In [ ]:
#### Annual Rates

scale = 10**6
geog = 'sub-region'

for label in np.unique(HI_STATS[geog]):
    label = label
    data = HI_STATS[HI_STATS[geog] == label]
    
    #### Rate of change
    coef, r2, p = lm_func(data, 'people_days')
    print('annual increase in people days '+label, 'was', coef/scale, ' p=', p)
    coef1, r21, p1 = lm_func(data, 'people_days_heat')
    print('annual increase in people days heat '+label, 'was', coef1/scale, ' p=', p)
    coef2, r22, p2 = lm_func(data, 'people_days_pop')
    print('annual increase in people days pop '+label, 'was', coef2/scale, ' p=', p)
    print('attrib heat '+label, 'was', coef1 / coef *100, ' p=', p, '\n')
  

In [ ]:
#### Trends for Africa, N & SS
geog = 'region'
location = 'Africa'
data = HI_STATS[HI_STATS[geog] == location]
print(location)

#### Total Change in people Days
data = data.groupby('year')['people_days'].sum()
year = str(data.index[33])
value = str(data.values[33]/10**9)
print('person days in 2016 was '+value+' billion')

year = str(data.index[0])
value = str(data.values[0]/10**9)
print('person days in 1983 was '+value+' billion')

#### Pct Change in Poeple Days 1983 - 2016
pdays16 = data.iloc[len(data) -1]
pdays83 = data.iloc[0]
out = (data.iloc[len(data) -1] - data.iloc[0]) / data.iloc[0] * 100
print('pct increase in people days 83 - 16 is ', out)



In [ ]:
#### S Asia as pct of total  global = 5.245146271 B 

print('pct of total pdays from S Asia is ', 1899.70765 / 10**3 / 5.245146271 * 100)

In [ ]:
#### Median Slope
region = 'Europe'
col = 'coef_heat'
geog = 'region'
scale = 10**3
result = city_coefs[city_coefs[geog]== region][col].median()
print(region, col, 'is ', result/scale)

# Heat Waves

- 9691 Kolkata 1998
- 2046 Paris 2003
- 4417, Aleppo 2010

In [ ]:
def make_data(dir_in, geog, location):
    """Function makes data to plot daily city-level HI Max and average
    Args:
        dir_in = directory to get data
        geog = column for geography, city-level = 'ID_HDC_G0'
        location = usually a city id
    """
    
    fn_list = sorted(glob.glob(dir_in+'*.csv')) # get data
    df_out = pd.DataFrame() # to write dataframe
    
     # get leap year cols from 2016
    hi16 = pd.read_csv(fn_list[33]) 
    cols = list(hi16.iloc[:,9:].columns)
    cols = [year[5:] for year in cols] # cols for data frame
    
    temp_list = [] # empty list for temps
    
    # loop through dir and get data
    for i, fn in enumerate(fn_list):
        df = pd.read_csv(fn) # open data frame
        year_label = [(df.columns[9]).split('.')[0]] # get year
        row = df[df[geog] == location]
        temp = row.iloc[:,9:] # get only temp columns
        
        # add in col for leap years
        if temp.shape[1] == 365:
            temp.insert(loc = 59, column = year_label[0]+'.02.29', value = np.nan, allow_duplicates=False)

        # Set Index & Columns
        temp.index = year_label
        temp.columns = cols # revalue to m.d
    
        # add to list
        temp_list.append(temp)
    
    df_out = pd.concat(temp_list) # make one big dataframe
    
    return df_out

def plot_data(df, year, start, end):#, start, end):
    """ Make the data for a plot
    Args: 
        df = df w/ daily HI max for a given city
        year = year you want to plot against average
        start = start of plot in julian days (e.g 1 - 365/366)
        end = end of plot in julian days
    """

    # Deal with leap year
    if year % 4 !=0:
        df.drop(columns ='02.29', inplace = True)
    
    # Subset data
    start = start - 1 # zero indexing 
    subset = df.iloc[:,start:end]
    
    # HI Max for year
    hi_year = subset.loc[str(year)]
    
    # make 34-avg daily hi and std
    means = subset.mean(axis = 0)
    stds = subset.std(axis = 0)
    
    # make colums to date time
    cols = pd.to_datetime([str(year)+'.'+date for date in hi_year.index])
    
    return hi_year, means, cols, stds

In [ ]:
# Find Heat Wave From All DATA
def select_city_year(df, city_id, year):
    "Quick search to find city and years within HI_STATS"
    df_out = df[(df['ID_HDC_G0'] == city_id) & (df['year'] == year)]
    
    return df_out

In [ ]:
# open heat events data
events_fn = os.path.join('','himax406_STATS.json')
events = pd.read_json(events_fn, orient = 'split')

In [ ]:
# [4417, 'Aleppo', 2010]  [9691, 'Kolkata', 1998] 
city = select_city_year(events, 4417, 2010)
city

In [ ]:
# Select event and compare to long term means
# Kolkata 1998 UID-3010504, Aleppo 2010 UID-984387 and UID-984386
event_id = 'UID-984386'
event = city[city['UID'] == event_id]
tmax = list(event['tmax'])[0]
dates = list(event['event_dates'])[0]

In [ ]:
# Look at event tmax 
max(tmax)

In [ ]:
# Look at event dates 
len(dates)

In [ ]:
#### Heat Index Data
data = 'himax'
data_label = '$WBGT_{max}$ '
DATA_IN = os.path.join('','GHS-'+data+'/') # output from avg wbgt/hi max
FIG_OUT = ''
DS = u"\N{DEGREE SIGN}"
t = 40.6 # wbgt (30) or hi (46.1) threshold

In [ ]:
# Get long term means
# Args
#[4417, 'Aleppo'] 2010 [2046, 'Paris'] 2003 [9691, 'Kolkata'] 1998 ['6955, Dehli']

# Args
city_list = [4417, 'Aleppo']
year = 2010

# April 1 to Sep 30 (Use Julian Days), or 1 - 182 lagos
start = 91 
end = 273

# Make Data 
df = make_data(DATA_IN, 'ID_HDC_G0', city_list[0])
years, means, cols, stds = plot_data(df, year, start, end)

In [ ]:
# what is the rank of HImax in the entire record?
sorted(df.to_numpy().flatten(), reverse = True)

In [ ]:
cols

In [ ]:
# make a df
df = pd.DataFrame()
df['dates'] = cols
df['max']  = years.to_list()
df['avg'] = means.to_list()
df['dif'] = df['max'] - df['avg']

In [ ]:
df

In [ ]:
type(df['dates'][0])

In [ ]:
type(dates[0][0])

In [ ]:
# start and end of heat wave
start = dates[0]
end = dates[-1]

In [ ]:
out = df[(df['dates'] >= start) & (df['dates'] <= end)]

In [ ]:
# peak heat wave
out.max()

In [ ]:
out[out['dif'] == out['dif'].max()]

In [ ]:
for dif in df['dif']:
    print(dif)

In [ ]:
df[df['dif'] > 8]

In [ ]:
len(df)

# Phoenix Caclulations
1990-6-26 Phoenix, AZ Weather History. https://www.wunderground.com/history/daily/KPHX/date/1990-6-26. Weather Underground. (June 10, 2021).<br>A. Minkler, At 118 degrees, Thursday heat in Phoenix breaks daily record set in 1934. The Arizona Republic (2020) (June 18, 2021).


In [ ]:
# Functions
def c_to_f(C):
    "Convert HI C to F"
    return 1.8 * C + 32 

def hi_to_wbgt(HI):
    """ Convert HI to WBGT using emprical relationship from Bernard and Iheanacho 2015
    WBGT [◦C] = −0.0034 HI2 + 0.96 HI−34; for HI [◦F]
    """
    
    WBGT = -0.0034*HI**2 + 0.96*HI - 34
    
    return WBGT

In [ ]:
# tmax was 122, rh was 11 using https://www.wpc.ncep.noaa.gov/html/heatindex.shtml
# then HI was 49 

# Check out cities

In [ ]:
def select_city_year(df, city_id, year):
    "Quick search to find city and years within HI_STATS"
    df_out = df[(df['ID_HDC_G0'] == city_id) & (df['year'] == year)]
    
    return df_out


In [ ]:
city = select_city_year(ALL_DATA, 4417, 2010)
city